In [15]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
path = "C:/Windows/Fonts/H2MJRE.TTF"     # matplotlib 한국어 설정
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)
from selenium import webdriver
import urllib.request
import urllib.parse
import time
import re #정규식

1. `url = 'https://everytime.kr/timetable'`: 접근할 사이트의 URL을 변수에 저장합니다.

2. `driver = webdriver.Chrome()`: Chrome 웹 드라이버를 초기화합니다. Chrome 웹 브라우저를 자동화하기 위해 사용됩니다. 이를 위해 Selenium WebDriver가 필요하며, 해당 라인은 WebDriver를 초기화하는 코드입니다.

3. `driver.get(url)`: 웹 드라이버를 사용하여 지정한 URL의 웹 페이지에 접근합니다.

4. `time.sleep(3)`: 3초간의 지연을 추가합니다. 웹 페이지가 완전히 로드될 때까지 기다리기 위해 사용됩니다. `time` 모듈이 임포트되어 있어야 합니다.

5. `user_id = input('에브리타임 아이디를 입력해주세요.: ')`: 사용자로부터 에브리타임 아이디를 입력받습니다.

6. `et_login = driver.find_element(By.NAME, "userid")`: `driver`를 사용하여 웹 페이지에서 "userid"라는 이름의 요소를 찾습니다. 에브리타임 로그인 폼에서 아이디 입력란을 가리킵니다.

7. `et_login.clear()`: 입력 요소의 내용을 지웁니다. 기존에 입력되어 있을 수 있는 텍스트를 제거합니다.

8. `et_login.send_keys(user_id)`: 입력 요소에 `user_id` 값을 입력합니다. 사용자가 입력한 에브리타임 아이디를 해당 요소에 보냅니다.

9. `user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')`: 사용자로부터 에브리타임 비밀번호를 입력받습니다.

10. `et_login = driver.find_element(By.NAME, "paasword")`: `driver`를 사용하여 웹 페이지에서 "paasword"라는 이름의 요소를 찾습니다. 에브리타임 로그인 폼에서 비밀번호 입력란을 가리킵니다.

11. `et_login.clear()`: 입력 요소의 내용을 지웁니다. 기존에 입력되어 있을 수 있는 텍스트를 제거합니다.

12. `et_login.send_keys(user_pw)`: 입력 요소에 `user_pw` 값을 입력합니다. 사용자가 입력한 에브리타임 비밀번호를 해당 요소에 보냅니다.

In [ ]:
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome()   # 주소 지정 없이
driver.get(url)
time.sleep(3)
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "userid")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "paasword")
et_login.clear()
et_login.send_keys(user_pw) 

driver.find_element(By.XPATH,"""//*[@id="container"]/form/p[3]/input""").click()
time.sleep(3)
while True:
    try:
        driver.find_element(By.XPATH,"""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
timetable_all = pd.DataFrame()
timetable_all
graph = {}
while True:
    try:
        driver.find_element(By.XPATH,"""//*[@id="container"]/ul/li[1]""").click()
        time.sleep(2)
        break
    except:
        continue
for i in range(9):
    while True:
        try:
            driver.find_element(By.XPATH,"""//*[@id="subjects"]/div[1]/a[4]""").click()
            time.sleep(2)
            break
        except:
            continue
    while i == 0:
        try:
            driver.find_element(By.XPATH,"""//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
            time.sleep(2)
            break
        except:
            continue
    while True:
        try:
            driver.find_element(By.XPATH,"""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li["""+str((i+1))+"""]""").click()
            time.sleep(2)
            break
        except:
            continue
#     while True:
#         try:
#             driver.find_element(By.XPATH,"""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/ul["""+str((i+1))+"""]/li""").click()
#             time.sleep(2)
#             break
#         except:
#             continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []
    tmp = soup.find('div','list').find('tbody').find_all('td')
    for i in tmp:
        tmp = i.get_text()
        contents.append(tmp)
    contents = np.array(contents)
    column = []
    tmp = tmp = soup.find('div','list').find('thead').find_all('th')
    for i in tmp:
        tmp = str(i).lstrip('<th>').split('<div>')
        column.append(tmp[0])
    timetable = pd.DataFrame(contents.reshape(len(contents)//12,12), columns=column)
    del(timetable['계획서']) ;del(timetable['강의평'])
    major = soup.find('a','item active').get_text().split(':')[1]
    graph[str(major)] = len(timetable)
    timetable_all = pd.concat([timetable_all,timetable],ignore_index=True)
    # 학과별 저장\
    timetable.to_excel('C:/Users/황의지' + major + 'everytime_timeschedule.xlsx', encoding='utf8', sheet_name=major, index=False)

# 전체 강좌 데이터 저장 (원하는 경로와 파일명으로 수정해주세요)
# timetable_all.to_excel('경로/19년 1학기 공과대학 강좌.xls', encoding='utf8', sheet_name='공과대학', index=False)

driver.close()

In [ ]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

driver = webdriver.Chrome() #주소 지정 없이

# 에브리타임 시간표 페이지로 이동
url = 'https://everytime.kr/timetable'
driver.get(url)
time.sleep(3)

# 에브리타임 로그인 정보 입력
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "userid")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "password")
et_login.clear()
et_login.send_keys(user_pw)

# 로그인 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click()
time.sleep(3)

# 시간표 페이지로 이동
while True:
    try:
        driver.find_element(By.XPATH, '//*[@id="sheet"]/ul/li[3]/a').click()
        time.sleep(2)
        break
    except:
        continue

timetable_all = pd.DataFrame()
graph = {}

# 각 학과의 강좌 크롤링
while True:
    try:
        driver.find_element(By.XPATH, '//*[@id="container"]/ul/li[1]').click()
        time.sleep(2)
        break
    except:
        continue

for i in range(9):
    while True:
        try:
            driver.find_element(By.XPATH, '//*[@id="subjects"]/div[1]/a[4]').click()
            time.sleep(2)
            break
        except:
            continue

    while i == 0:
        try:
            driver.find_element(By.XPATH, '//*[@id="subjectCategoryFilter"]/div/ul/li[2]').click()
            time.sleep(2)
            break
        except:
            continue

    while True:
        try:
            driver.find_element(By.XPATH, '//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li[' + str(i+1) + ']').click()
            time.sleep(2)
            break
        except:
            continue

    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []
    tmp = soup.find('div', 'list').find('tbody').find_all('td')
    for i in tmp:
        tmp = i.get_text()
        contents.append(tmp)
    contents = np.array(contents)

    column = []
    tmp = soup.find('div', 'list').find('thead').find_all('th')
    for i in tmp:
        tmp = str(i).lstrip('<th>').split('<div>')
        column.append(tmp[0])

    timetable = pd.DataFrame(contents.reshape(len(contents)//12, 12), columns=column)
    del(timetable['계획서'])
    del(timetable['강의평'])
    major = soup.find('a', 'item active').get_text().split(':')[1]
    graph[str(major)] = len(timetable)
    timetable_all = pd.concat([timetable_all, timetable], ignore_index=True)

    # 엑셀 파일로 저장 (원하는 경로와 파일명으로 수정해주세요)
    timetable.to_excel('C:/Users/황의지' + major + 'everytime_timeschedule.xlsx', encoding='utf8', sheet_name=major, index=False)

# 전체 강좌 데이터 저장 (원하는 경로와 파일명으로 수정해주세요)
# timetable_all.to_excel('경로/19년 1학기 공과대학 강좌.xls', encoding='utf8', sheet_name='공과대학', index=False)

time.sleep(2)
driver.close()
